In [1]:
# import libraries

import pandas as pd
import os

<h3> Task 3 - Data management:

1. Locating the .csv files in the Task 3 & 4 folder.

In [2]:
# Defining the folder path to locate the CSV files
folder_path = 'Task 3 _ 4'

# Get all files in the folder
files = os.listdir(folder_path)

# Filter .csv files
csv_files = [file for file in files if file.endswith('.csv')]

# Print the list of .csv files
for csv_file in csv_files:
    print(csv_file)

customer_database.csv
data_dictionary.csv
merged_database.csv
orders_database.csv
test_set.csv
training_set.csv


There are only three files in the Task 3 & 4 folder.
- customer_database.csv
- data_dictionary.csv
- orders_database.csv

2. Merging the two databases customer_database.csv and orders_database.csv into one table.

In [3]:
# Read the CSV files into A Pandas DataFrame
customer_database=pd.read_csv('Task 3 _ 4\customer_database.csv')
orders_database=pd.read_csv('Task 3 _ 4\orders_database.csv')

In [4]:
# summary for customer data 
customer_database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Name                  9 non-null      object
 1   Age                   9 non-null      int64 
 2   Height                9 non-null      int64 
 3   Sex                   9 non-null      object
 4   Location              9 non-null      object
 5   Income in May         9 non-null      int64 
 6   Income in June        9 non-null      int64 
 7   Income in July        9 non-null      int64 
 8   Water bill (monthly)  9 non-null      int64 
dtypes: int64(6), object(3)
memory usage: 776.0+ bytes


From the above summary we can tell;
- The total number of columns in customer database and their names.
- Total number of rows and if there are nulls
- Data type for each column

In [5]:
# print sample customer_database
customer_database.sample(3)

,Name,Age,Height,Sex,Location,Income in May,Income in June,Income in July,Water bill (monthly)
2,Peter,42,167,M,A,1300,1320,1350,140
4,Stella,33,160,F,A,955,1050,1000,91
8,Lynn,46,190,F,B,1200,1250,1200,112


In [6]:
# summary for orders data
orders_database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Name                   9 non-null      object
 1   Number of orders May   9 non-null      int64 
 2   Number of orders June  9 non-null      int64 
 3   Number of orders July  9 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 416.0+ bytes


From the above summary we can tell;
- The total number of columns in orders database and their names.
- Total number of rows and if there are nulls.
- Data type for each column.

In [7]:
# print sample orders_database
orders_database.sample(3)

,Name,Number of orders May,Number of orders June,Number of orders July
0,Juan,203,223,210
4,Stella,188,209,199
1,Julia,251,240,239


*Based on the above summary, below are the steps to approach this task.*

Steps.

1. Create income and Month column in customer database (extract from 'Income in May', 'Income in June', 'Income in July').
2. Extract Month and Number of orders in orders_database.
3. Merge the two DataFrames.
4. Replace the human name column headings with their respective variable names.
5. Combining income and num_orders into single column.
6. Explore the merged data.
7. Save merged data as a CSV file.

In [9]:
def adding_columns(df, id_vars):

    '''A function to help in creating the new columns, Month, Income, Number of orders in
    the customer database and the orders database.
    '''
    #Reshape (Unpivot) the DataFrame using melt function
    df = df.melt(id_vars=id_vars, var_name='Month', value_name='Income')
    
    # Extract the month by splitting the column values on whitespace and retrieving the last word
    df['Month'] = df['Month'].str.split().str[-1]
    
    return df

- Apply the function on customer database to extract Month and Income values

In [10]:
# Apply the function on customer database to extract Month and Income values
customer_db = adding_columns(customer_database, ['Name', 'Age', 'Height','Sex','Location','Water bill (monthly)'])

# Display the updated dataframe
customer_db.head(3)

,Name,Age,Height,Sex,Location,Water bill (monthly),Month,Income
0,Juan,51,170,M,A,110,May,1000
1,Julia,20,187,F,B,119,May,1250
2,Peter,42,167,M,A,140,May,1300


- Apply the function on orders_database to create Month and number of orders

In [11]:
# Apply the function on orders_database to extract Month and number of orders
order_db = adding_columns(orders_database, 'Name')

# rename column income to Number of orders
order_db.rename(columns={"Income":"Number of orders"}, inplace=True)

# Display the updated dataframe
order_db.head(3)

,Name,Month,Number of orders
0,Juan,May,203
1,Julia,May,251
2,Peter,May,255


- Merging the databases above (customer_db and orders_db)

In [12]:
# Outer merge customer db and orders db to return customers from both databases.
merged_database= pd.merge(customer_db[['Name','Age','Height','Location','Water bill (monthly)','Month','Income']],
                          order_db,on=['Name','Month',], how='outer')
merged_database.head(3)

,Name,Age,Height,Location,Water bill (monthly),Month,Income,Number of orders
0,Juan,51,170,A,110,May,1000,203
1,Julia,20,187,B,119,May,1250,251
2,Peter,42,167,A,140,May,1300,255


- Replacing the human name heading with respective variable name

In [13]:
# Replace column names with lowercase letters to match respective variable names
merged_database.columns = merged_database.columns.str.lower()

#renaming columns water bill (monthly) and number of orders
merged_database.rename(columns={'water bill (monthly)': 'water_bill', 'number of orders': 'num_orders'}, inplace=True)

# Print the updated DataFrame
merged_database.head()

,name,age,height,location,water_bill,month,income,num_orders
0,Juan,51,170,A,110,May,1000,203
1,Julia,20,187,B,119,May,1250,251
2,Peter,42,167,A,140,May,1300,255
3,John,32,190,C,91,May,800,163
4,Stella,33,160,A,91,May,955,188


- Combining income and number of orders to single column separated by | where the first value is income and last is the  number of orders.

In [14]:
# combining income and number of orders into single column
merged_database['income|num_orders'] = merged_database.apply(lambda row: f"{row['income']}|{row['num_orders']}", axis=1)

# drop income and num_orders column
merged_database.drop(columns=['income','num_orders'], inplace=True)

merged_database.head(3)

,name,age,height,location,water_bill,month,income|num_orders
0,Juan,51,170,A,110,May,1000|203
1,Julia,20,187,B,119,May,1250|251
2,Peter,42,167,A,140,May,1300|255


- Exploring merged data 

In [15]:
# summary of the merged data
merged_database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               27 non-null     object
 1   age                27 non-null     int64 
 2   height             27 non-null     int64 
 3   location           27 non-null     object
 4   water_bill         27 non-null     int64 
 5   month              27 non-null     object
 6   income|num_orders  27 non-null     object
dtypes: int64(3), object(4)
memory usage: 1.6+ KB


In [16]:
# saving data to Task3_4 folder
merged_database.to_csv('Task 3 _ 4/merged_database.csv', index=False)